# Assignment 1: Time Series Forecast With Python (Seasonal ARIMA)

**Lecturer**: Vincent Claes<br>
**Authors:** Bryan Honof, Jeffrey Gorissen<br>
**Start Date:** 19/10/2018
    
**Objective:** Visualize and predict the future temperatures via ARIMA

**Description:** In this notebook we train our model

In [1]:
import math
import warnings
import datetime

import pandas            as pd
import itertools         as it
import statsmodels.api   as sm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore") # specify to ignore warning messages

In [2]:
data_csv = pd.read_csv('./data/rolmean_data.csv')
data = pd.DataFrame()

# Convert the creation_date column to datetime64
data['dateTime'] = pd.to_datetime(data_csv['dateTime'])
# Convert the value column to float
data['temperature'] = pd.to_numeric(data_csv['temperature'])

# Set the dateTime column as index
data = data.set_index(['dateTime'])

# Sort the dataFrame just to be sure...
data = data.sort_index()

data = data.dropna()

# Double check the results
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 845 entries, 2018-11-11 03:00:00 to 2018-11-19 22:00:00
Data columns (total 1 columns):
temperature    845 non-null float64
dtypes: float64(1)
memory usage: 13.2 KB


In [3]:
data.tail(5)

,temperature
dateTime,
2018-11-19 21:00:00,16.77
2018-11-19 21:15:00,16.89
2018-11-19 21:30:00,17.05
2018-11-19 21:45:00,17.17
2018-11-19 22:00:00,17.24


## Search for best parameters


```p``` is the auto-regressive part of the model. It allows us to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to be warm tomorrow if it has been warm the past 3 days.<br>
```d``` is the integrated part of the model. This includes terms in the model that incorporate the amount of differencing (i.e. the number of past time points to subtract from the current value) to apply to the time series. Intuitively, this would be similar to stating that it is likely to be same temperature tomorrow if the difference in temperature in the last three days has been very small.<br>
```q``` is the moving average part of the model. This allows us to set the error of our model as a linear combination of the error values observed at previous time points in the past.

In [4]:
# Tune Seasonal ARIMA model
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

# Generate all different combinations of p, q and q triplets
pdq = list(it.product(p, d, q))
print(pdq)

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 96) for x in list(it.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 4)
SARIMAX: (0, 0, 1) x (0, 1, 0, 4)
SARIMAX: (0, 1, 0) x (0, 1, 1, 4)
SARIMAX: (0, 1, 0) x (1, 0, 0, 4)


In [5]:
result_list = []
AIC         = []
parm_       = []
parm_s      = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            AIC.append(results.aic)
            parm_.append(param)
            parm_s.append(param_seasonal)
            
            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, round(results.aic,2)))
            result_list.extend([param, param_seasonal, round(results.aic,2)])
        except:
            print('error')
            continue
            
print('Done!')

ARIMA(0, 0, 0)x(0, 0, 0, 4) - AIC:7299.49
ARIMA(0, 0, 0)x(0, 0, 1, 4) - AIC:6128.16
ARIMA(0, 0, 0)x(0, 1, 0, 4) - AIC:723.58
ARIMA(0, 0, 0)x(0, 1, 1, 4) - AIC:312.84
ARIMA(0, 0, 0)x(1, 0, 0, 4) - AIC:724.95
ARIMA(0, 0, 0)x(1, 0, 1, 4) - AIC:328.79
ARIMA(0, 0, 0)x(1, 1, 0, 4) - AIC:147.92
ARIMA(0, 0, 0)x(1, 1, 1, 4) - AIC:136.28
ARIMA(0, 0, 1)x(0, 0, 0, 4) - AIC:6137.2
ARIMA(0, 0, 1)x(0, 0, 1, 4) - AIC:4974.86
ARIMA(0, 0, 1)x(0, 1, 0, 4) - AIC:-303.87
ARIMA(0, 0, 1)x(0, 1, 1, 4) - AIC:-538.08
ARIMA(0, 0, 1)x(1, 0, 0, 4) - AIC:-300.6
ARIMA(0, 0, 1)x(1, 0, 1, 4) - AIC:-528.72
ARIMA(0, 0, 1)x(1, 1, 0, 4) - AIC:-655.02
ARIMA(0, 0, 1)x(1, 1, 1, 4) - AIC:-653.79
ARIMA(0, 1, 0)x(0, 0, 0, 4) - AIC:-1266.24
ARIMA(0, 1, 0)x(0, 0, 1, 4) - AIC:-1390.54
ARIMA(0, 1, 0)x(0, 1, 0, 4) - AIC:-1188.36
ARIMA(0, 1, 0)x(0, 1, 1, 4) - AIC:-1317.44
ARIMA(0, 1, 0)x(1, 0, 0, 4) - AIC:-1454.66
ARIMA(0, 1, 0)x(1, 0, 1, 4) - AIC:-1462.57
ARIMA(0, 1, 0)x(1, 1, 0, 4) - AIC:-1309.76
ARIMA(0, 1, 0)x(1, 1, 1, 4) - AIC:-

In [6]:
min(AIC)
pos = AIC.index(min(AIC))
print(parm_[pos], parm_s[pos], min(AIC))

(1, 1, 1) (0, 0, 0, 4) -1720.4721632496312


[next notebook](./5_fitting_and_predicting.ipynb)